# Resiliency tool

In [ ]:
import reXplan as rx
import pandas as pd
import numpy as np
from datetime import date as dt_date

from pandapower.plotting.plotly import simple_plotly, vlevel_plotly, pf_res_plotly
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pandapower.plotting.plotly import simple_plotly, vlevel_plotly, pf_res_plotly
from utils import * # pplotting functions

import warnings
warnings.simplefilter("ignore") # warning are ignored for now


## Network initialization

In [3]:
simulationName = 'Simbench';
network = rx.network.Network(simulationName);
simulation = rx.simulation.Sim(simulationName);

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
 72% (8 of 11) |##################       | Elapsed Time: 0:00:00 ETA:  00:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Simulation: First timestep = 1, last timestep= 51
Simulation: First timestep = 2, last timestep= 7


## CLOSE

#### Fragility Curves are automatically generated once the network object is created

### Plot a specific fragility curve:

In [ ]:
xnew = np.linspace(0, 80, num=1000, endpoint=True)
fig, ax = network.fragilityCurves['towers_1'].plot_fc(xnew)

### Or Plot all the fragility curves in the database

In [ ]:
fig, ax = rx.fragilitycurve.plotFragilityCurves(network.fragilityCurves, xnew)

### Method 1: Generate a hazard element by reading a .nc file

In [ ]:
network.event.hazardFromNC('sythetic_data.nc')
fig, ax = network.event.plot(10, projection='cyl', edge_pad=0)

In [ ]:
network.event.plot_gif('sythetic_data.gif', speed=3, projection='cyl', edge_pad=0)

![method1](file/input/basic_example/hazards/gif/sythetic_data.gif "method1")

### Method 2: Generate a hazard element by reading a trajectory csv file

In [ ]:
network.event.hazardFromTrajectory('trajectory.csv', 
                                   max_intensity=60., max_radius=50., 
                                   sdate = dt_date(2022,4,1), edate = dt_date(2022,4,3), 
                                   geodata1 = rx.network.GeoData(47.4,5.8), geodata2 = rx.network.GeoData(54.9,15.0),
                                   delta_km=10, frequency='1H')

fig, ax = network.event.plot(10, projection='cyl', edge_pad=0)

In [ ]:
network.event.plot_gif('trajectory.gif', speed=3, projection='cyl', edge_pad=0)

![method2](file/input/basic_example/hazards/gif/trajectory.gif "method2")

### Method 3: Generate a static hazard element by providing the location of the epicenter

In [ ]:
network.event.hazardFromStaticInput('static_event.nc',
                                    max_intensity=90, max_radius=100,
                                    sdate = dt_date(2022,4,1), edate = dt_date(2022,4,3),
                                    geodata1 = rx.network.GeoData(47.4,5.8), geodata2 = rx.network.GeoData(54.9,15.0),
                                    delta_km=10,
                                    epicenter_lat=50, epicenter_lon=9,
                                    frequency='1H', epicenter_radius=1, epicenter_intensity=1)

fig, ax = network.event.plot(10, projection='cyl', edge_pad=0)

In [ ]:
network.event.plot_gif('static_event.gif', speed=3, projection='cyl', edge_pad=0)

![method3](file/input/basic_example/hazards/gif/static_event.gif "method3")

### For Methods 1-3: Use simulation.initialize_model_sh to generate outage schedule

In [ ]:
simulation.initialize_model_sh(network, iterationNumber=10)
simulation.run(network,
                #iterationSet = [1],
                run_type = 'pm_ac_opf', 
                delta = 1e-16, 
                saveOutput = True)

In [ ]:
pf_res_plotly(network.pp_network);

In [ ]:
simulation.results.loc[:,:,:,'network',:]

## Method 4: Simulate multiple events according a given return period

In [4]:
simulation.initialize_model_rp(network=network, ref_return_period="rp1", iterationNumber=2, maxTotalIteration=1000, cv=0.1, nStrataSamples=10000)


x_min = 10
x_max = 70

Strata  3
Sample size  168
Strata  4
Sample size  89
Strata  5
Sample size  51
Strata  6
Sample size  33
Strata  7
Sample size  27
Strata  8
Sample size  23
Strata  9
Sample size  23
Strata  10
Sample size  23
-----------------
 Kmeans solution 
-----------------
 *** Domain:  1  ***
 Number of strata:  10
 Sample size     :  23
Computations are being done on population data

Number of strata:  10
... of which with only one unit:  0
Input data have been checked and are compliant with requirements

 *** Domain :  1   1
 Number of strata :  10000
 *** Sample cost:  15.11226
 *** Number of strata:  7
 *** Sample size :  15
 *** Number of strata :  7
---------------------------
Strata = 0
Number of samples = 4.0
Intensity samples between 9.315312296042517 and 37.556706832852896

Strata = 1
Number of samples = 4.0
Intensity samples between 37.56199177762471 and 46.511994452085084

Strata = 2
Number of samples = 6.0
Intensity samples between 46.51755326557219 and 53.59

### CLOSE

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
for rp in network.returnPeriods.keys():
    sns.lineplot(x=network.returnPeriods[rp].x_data, y=network.returnPeriods[rp].y_data)

In [ ]:
simulation.stratResults

In [ ]:
plt.hist(simulation.samples, density=True, bins=20)
for b in np.append(simulation.stratResults["Upper_X1"].values, simulation.stratResults["Lower_X1"].values[0]):
    plt.axvline(x = b, color = 'r')

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

df = simulation.failureProbs[simulation.failureProbs['element type']=='Line']
sns.lineplot(data=df, x='event intensity', y='failure probability', hue='power element')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
simulation.failureProbs[(simulation.failureProbs['element type']=='Generator') & (simulation.failureProbs['iteration']==4)]

## Launching montecarlo simulations
Optimal power flow (40 steps) over 8 montercalo iterations divided into 4 stratas.

In [1]:
import reXplan as rx
import pandas as pd
import numpy as np
from datetime import date as dt_date

from pandapower.plotting.plotly import simple_plotly, vlevel_plotly, pf_res_plotly
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from pandapower.plotting.plotly import simple_plotly, vlevel_plotly, pf_res_plotly
from utils import * # pplotting functions

import warnings
warnings.simplefilter("ignore") # warning are ignored for now
simulationName = 'Simbench';
network = rx.network.Network(simulationName);
simulation = rx.simulation.Sim(simulationName);

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Simulation: First timestep = 1, last timestep= 51
Simulation: First timestep = 2, last timestep= 7


In [2]:
time = rx.simulation.Time(timepoints = [1,2])
simulation.run(network, iterationSet = [0], time = time,  run_type = 'pm_ac_opf', delta = 1e-16, appendOutput=True)
time = rx.simulation.Time(timepoints = [1,2])
simulation.run(network, iterationSet = [1], time = time,  run_type = 'pm_ac_opf', delta = 1e-16, appendOutput=True)
time = rx.simulation.Time(timepoints = [3,4])
simulation.run(network, iterationSet = [1], time = time,  run_type = 'pm_ac_opf', delta = 1e-16, appendOutput=True)
time = rx.simulation.Time(timepoints = [5,6,7,8])
simulation.run(network, iterationSet = [2], time = time,  run_type = 'pm_ac_opf', delta = 1e-16, appendOutput=True)
time = rx.simulation.Time(timepoints = [1,7,8])
df = simulation.run(network, iterationSet = [3], time = time,  run_type = 'pm_ac_opf', delta = 1e-16, appendOutput=True)

Strata = 0; Iteration = 0


100%|██████████| 2/2 [00:31<00:00, 15.65s/it]


No output database found.
Saving output database...
done!
Strata = 0; Iteration = 1


100%|██████████| 2/2 [00:01<00:00,  1.93it/s]


Appending to output database...
Strata = 0; Iteration = 1


...

Appending to output database...
Strata = 0; Iteration = 2


100%|██████████| 4/4 [00:01<00:00,  2.96it/s]


Appending to output database...
Strata = 0; Iteration = 3


100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


Appending to output database...


In [3]:
df.head(10)

,strata,iteration,field,type,id,2022-01-01 01:00:00,2022-01-01 02:00:00,2022-01-01 03:00:00,2022-01-01 04:00:00,2022-01-01 05:00:00,2022-01-01 06:00:00,2022-01-01 07:00:00,2022-01-01 08:00:00
0,0,0,in_service,bus,EHV Bus 1865,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,in_service,bus,EHV Bus 1866,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,in_service,bus,EHV Bus 3092,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,in_service,bus,EHV Bus 3093,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,in_service,bus,EHV Bus 3094,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,in_service,bus,HV2 Bus 1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,0,0,in_service,bus,HV2 Bus 10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0,in_service,bus,HV2 Bus 100,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,0,0,in_service,bus,HV2 Bus 101,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,0,0,in_service,bus,HV2 Bus 102,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# time = rx.simulation.Time(start = 16, duration = 5)
#time = rx.simulation.Time(interval = [1,2,4,7,9,13])
simulation.run_prediction(network, run_type = 'pm_ac_opf', delta = 1e-16)

In [ ]:
simulation.results.loc[:,:,:,'network',:]

## Iterations metrics

In [ ]:
df = pd.read_csv(rx.config.path.engineDatabaseFile(simulationName), index_col = [0, 1, 2, 3, 4]) # read database with results
df = filter_non_converged_iterations(df) # filterining non-converged iterations


In [ ]:
df_line = group_by(df.loc[3], 'sum', 'iteration', 'field', 'type').loc[:,:,'line']
df_line_quantiles = invert(get_quantiles_on_iterations(df_line, [0.05,0.5,0.95]))
df_line = invert(df_line)

In [ ]:
df_montecarlo = pd.read_csv(rx.config.path.engineDatabaseFile(simulationName), index_col = [0, 1, 2, 3, 4])

### Number of lines in service

In [ ]:
px.line(df_line, x=df_line.index, y = 'in_service', color = 'iteration')

In [ ]:
px.line(df_line_quantiles, x=df_line_quantiles.index, y = 'in_service', color = 'quantile')

In [ ]:
df_load = group_by(filter(df, type = 'load'), 'sum', 'iteration', 'field', 'type')
df_load_quantiles = invert(get_quantiles_on_iterations(df_load, [0.05, 0.25, 0.5, 0.75, 0.95]))
# df_load = invert(df_load) 
# df_load['loss_of_load_p_percentage'] = (df_load['loss_of_load_p_mw'])/df_load['max_p_mw'] *100
df_load_quantiles['loss_of_load_p_percentage'] = (df_load_quantiles['loss_of_load_p_mw'])/df_load_quantiles['max_p_mw'] *100

In [ ]:
px.line(df_load_quantiles, x=df_load_quantiles.index, y = 'loss_of_load_p_percentage', color = 'quantile')

In [ ]:
df_network = invert(filter(df, type = 'network')) # filter network fields and invert for plotting
px.scatter(df_network, x=df_network.index, y= 'energy_not_served_mwh' )

## Montercalo metrics

In [ ]:
df_network_condensed = filter(df, type = 'network').sum(axis = 1) # sum over timesteps

In [ ]:
df_network_condensed_ = invert(df_network_condensed)
px.histogram(df_network_condensed_, x='energy_not_served_mwh', histnorm='probability')

In [ ]:
statistics= df_network_condensed.groupby('field').mean() # average over iterations
EENS = statistics['energy_not_served_mwh']
LOLE = statistics['loss_of_load_p_duration_h']
print(f'EENS : {EENS.round(2)} MWh, LOLE : {LOLE.round(2)} h')

## Survivability
Probability of supplying at minimum percentage of the load.

In [ ]:
crt_loss_of_load = 30 
df_loss_of_load = df.loc[:,:,"loss_of_load_p_percentage","network"]
Survivability = pd.DataFrame(1 - (df_loss_of_load > crt_loss_of_load).sum() / df_loss_of_load.index.levels[0].size, columns = ['base case'])

#df_aux = pd.read_csv(rx.config.path.engineDatabaseFile('basic_example_v1'), index_col = [0, 1, 2, 3, 4])
#df_loss_of_load_aux = df_aux.loc[:,"loss_of_load_p_percentage","network"]
#Survivability['line 10 reinforced'] = 1 - (df_loss_of_load_aux > crt_loss_of_load).sum() / df_loss_of_load_aux.index.levels[0].size

#df_aux = pd.read_csv(rx.config.path.engineDatabaseFile('basic_example_v2'), index_col = [0, 1, 2, 3, 4])
#df_loss_of_load_aux = df_aux.loc[:,"loss_of_load_p_percentage","network"]
#Survivability['line 2 reparing time improved'] = 1 - (df_loss_of_load_aux > crt_loss_of_load).sum() / df_loss_of_load_aux.index.levels[0].size

In [ ]:
px.line(Survivability).update_layout(xaxis_title="time", yaxis_title="Survivability")

In [ ]:
df_line = group_by(filter(df, type = 'line'), 'mean','strata', 'iteration', 'field','id') # mean in this case does not have any effect as the groupying levels are the initial ones
df_line = invert(df_line)
# df_line = df_line.loc[df_line.index > '2022-01-01 12:00:00']

fig = go.Figure() # --> put in a function (?)

ids = df_line['id'].drop_duplicates().to_list()

for id in ids:
    fig.add_trace(go.Violin(x=df_line['id'][df_line['id'] == id],
                            y=df_line['loading_percent'][df_line['id'] == id],
                            name=id,
                            box_visible=False,
                            meanline_visible=True,
                            side='positive',
                            orientation = 'v'
                           )
                 )
fig.update_layout(width=1000, height=500)
fig.show()

In [ ]:
df_bus =invert(filter(df, type = 'bus'))
fig = go.Figure()

ids = df_bus['id'].drop_duplicates().to_list()

for id in ids:
    fig.add_trace(go.Violin(x=df_bus['id'][df_bus['id'] == id],
                            y=df_bus['vm_pu'][df_bus['id'] == id],
                            name=id,
                            #box_visible=True,
                            meanline_visible=True,
                            side='positive',
                            orientation = 'v'
                           )     
                 )
fig.update_layout(width=1000, height = 500)
fig.show()